In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os
import time
from urllib.parse import urlparse

# 要爬的頁面清單,按照類別分組
URL_LIST = {
    "architecture": [
        "https://docs.streamlit.io/develop/concepts/architecture/run-your-app",
        "https://docs.streamlit.io/develop/concepts/architecture/architecture", 
        "https://docs.streamlit.io/develop/concepts/architecture/app-chrome",
        "https://docs.streamlit.io/develop/concepts/architecture/caching",
        "https://docs.streamlit.io/develop/concepts/architecture/session-state",
        "https://docs.streamlit.io/develop/concepts/architecture/forms",
        "https://docs.streamlit.io/develop/concepts/architecture/fragments",
        "https://docs.streamlit.io/develop/concepts/architecture/widget-behavior"
    ],
    
    "multipage": [
        "https://docs.streamlit.io/develop/concepts/multipage-apps/overview",
        "https://docs.streamlit.io/develop/concepts/multipage-apps/page-and-navigation",
        "https://docs.streamlit.io/develop/concepts/multipage-apps/pages-directory",
        "https://docs.streamlit.io/develop/concepts/multipage-apps/widgets"
    ],
    
    "design": [
        "https://docs.streamlit.io/develop/concepts/design/animate",
        "https://docs.streamlit.io/develop/concepts/design/buttons",
        "https://docs.streamlit.io/develop/concepts/design/dataframes",
        "https://docs.streamlit.io/develop/concepts/design/multithreading",
        "https://docs.streamlit.io/develop/concepts/design/custom-classes",
        "https://docs.streamlit.io/develop/concepts/design/timezone-handling"
    ],
    
    "connections": [
        "https://docs.streamlit.io/develop/concepts/connections/connecting-to-data",
        "https://docs.streamlit.io/develop/concepts/connections/secrets-management", 
        "https://docs.streamlit.io/develop/concepts/connections/authentication",
        "https://docs.streamlit.io/develop/concepts/connections/security-reminders"
    ],
    
    "custom_components": [
        "https://docs.streamlit.io/develop/concepts/custom-components/intro",
        "https://docs.streamlit.io/develop/concepts/custom-components/create",
        "https://docs.streamlit.io/develop/concepts/custom-components/publish",
        "https://docs.streamlit.io/develop/concepts/custom-components/limitations"
    ],
    
    "api_reference": {
        "text": [
            "https://docs.streamlit.io/develop/api-reference/write-magic/st.write",
            "https://docs.streamlit.io/develop/api-reference/write-magic/st.write_stream",
            "https://docs.streamlit.io/develop/api-reference/text/st.markdown",
            "https://docs.streamlit.io/develop/api-reference/text/st.badge",
            "https://docs.streamlit.io/develop/api-reference/text/st.code",
            "https://docs.streamlit.io/develop/api-reference/text/st.divider",
            "https://docs.streamlit.io/develop/api-reference/text/st.html"
        ],
        "data": [
            "https://docs.streamlit.io/develop/api-reference/data/st.dataframe",
            "https://docs.streamlit.io/develop/api-reference/data/st.data_editor",
            "https://docs.streamlit.io/develop/api-reference/data/st.table",
            "https://docs.streamlit.io/develop/api-reference/data/st.metric",
            "https://docs.streamlit.io/develop/api-reference/data/st.json"
        ],
        "column_config": [
            "https://docs.streamlit.io/develop/api-reference/data/st.column_config/st.column_config.checkboxcolumn",
            "https://docs.streamlit.io/develop/api-reference/data/st.column_config/st.column_config.selectboxcolumn",
            "https://docs.streamlit.io/develop/api-reference/data/st.column_config/st.column_config.areachartcolumn",
            "https://docs.streamlit.io/develop/api-reference/data/st.column_config/st.column_config.linechartcolumn",
            "https://docs.streamlit.io/develop/api-reference/data/st.column_config/st.column_config.barchartcolumn",
            "https://docs.streamlit.io/develop/api-reference/data/st.column_config/st.column_config.progresscolumn"
        ],
        "charts": [
            "https://docs.streamlit.io/develop/api-reference/charts/st.area_chart",
            "https://docs.streamlit.io/develop/api-reference/charts/st.bar_chart", 
            "https://docs.streamlit.io/develop/api-reference/charts/st.line_chart",
            "https://docs.streamlit.io/develop/api-reference/charts/st.map",
            "https://docs.streamlit.io/develop/api-reference/charts/st.scatter_chart",
            "https://docs.streamlit.io/develop/api-reference/charts/st.altair_chart",
            "https://docs.streamlit.io/develop/api-reference/charts/st.bokeh_chart",
            "https://docs.streamlit.io/develop/api-reference/charts/st.graphviz_chart",
            "https://docs.streamlit.io/develop/api-reference/charts/st.plotly_chart",
            "https://docs.streamlit.io/develop/api-reference/charts/st.pydeck_chart",
            "https://docs.streamlit.io/develop/api-reference/charts/st.pyplot",
            "https://docs.streamlit.io/develop/api-reference/charts/st.vega_lite_chart"
        ],
        "widgets": [
            "https://docs.streamlit.io/develop/api-reference/widgets/st.feedback",
            "https://docs.streamlit.io/develop/api-reference/widgets/st.button",
            "https://docs.streamlit.io/develop/api-reference/widgets/st.pills",
            "https://docs.streamlit.io/develop/api-reference/widgets/st.multiselect",
            "https://docs.streamlit.io/develop/api-reference/widgets/st.segmented_control",
            "https://docs.streamlit.io/develop/api-reference/widgets/st.toggle",
            "https://docs.streamlit.io/develop/api-reference/widgets/st.date_input",
            "https://docs.streamlit.io/develop/api-reference/widgets/st.time_input",
            "https://docs.streamlit.io/develop/api-reference/widgets/st.text_area",
            "https://docs.streamlit.io/develop/api-reference/widgets/st.file_uploader"
        ],
        "layout": [
            "https://docs.streamlit.io/develop/api-reference/layout/st.columns",
            "https://docs.streamlit.io/develop/api-reference/layout/st.container"
        ],
        "execution_flow": [
            "https://docs.streamlit.io/develop/api-reference/execution-flow/st.dialog",
            "https://docs.streamlit.io/develop/api-reference/execution-flow/st.form",
            "https://docs.streamlit.io/develop/api-reference/execution-flow/st.form_submit_button",
            "https://docs.streamlit.io/develop/api-reference/execution-flow/st.fragment",
            "https://docs.streamlit.io/develop/api-reference/execution-flow/st.rerun",
            "https://docs.streamlit.io/develop/api-reference/execution-flow/st.stop"
        ],
        "caching": [
            "https://docs.streamlit.io/develop/api-reference/caching-and-state/st.cache_data",
            "https://docs.streamlit.io/develop/api-reference/caching-and-state/st.cache_resource",
            "https://docs.streamlit.io/develop/api-reference/caching-and-state/st.context",
            "https://docs.streamlit.io/develop/api-reference/caching-and-state/st.query_params"
        ],
        "connections": [
            "https://docs.streamlit.io/develop/api-reference/connections/st.connections.sqlconnection",
            "https://docs.streamlit.io/develop/api-reference/connections/st.connection"
        ],
        "custom_components": [
            "https://docs.streamlit.io/develop/api-reference/custom-components/st.components.v1.html",
            "https://docs.streamlit.io/develop/api-reference/custom-components/st.components.v1.iframe"
        ],
        "configuration": [
            "https://docs.streamlit.io/develop/api-reference/configuration/config.toml",
            "https://docs.streamlit.io/develop/api-reference/configuration/st.get_option",
            "https://docs.streamlit.io/develop/api-reference/configuration/st.set_page_config",
            "https://docs.streamlit.io/develop/api-reference/configuration/st.set_option"
        ]
    }
}


SAVE_ROOT = "/Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit"
CRAWL_DELAY = 1.5


def scrape_main_content_as_markdown(url, save_path):
    """從單一頁面爬主內文，轉成 markdown 儲存，並跳過導航部分"""
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    lines = [
        "---",
        f"title: Streamlit Documentation Capture",
        f"url: {url}",
        f"date: {datetime.now():%Y-%m-%d %H:%M:%S}",
        "---\n",
    ]

    main = soup.find("main")
    if not main:
        print(f"⚠️ 找不到 main: {url}")
        return
        
    # 用於標記是否已經找到真正的內容區
    content_started = False
    # 用於記錄可能的主標題
    main_heading = None
    
    # 先找到第一個真正的主標題（這通常是 h1 標籤，且不包含導航內容）
    all_elements = main.find_all(["h1", "h2", "h3", "h4", "h5", "h6", "p", "pre", "code", "ul", "ol", "li"])
    
    for el in all_elements:
        # 尋找主標題，這通常是實際內容的開始
        if el.name == "h1" and not content_started:
            text = el.get_text(strip=True)
            # 避免捕獲像 "Documentation" 或空標題這樣的導航元素
            if text and not text.lower() == "documentation" and not el.find_parent("nav"):
                main_heading = el
                content_started = True
                lvl = int(el.name[1])
                lines.append(f"{'#' * lvl} {text}\n")
                break
    
    # 如果找不到 h1，嘗試尋找 h2
    if not content_started:
        for el in all_elements:
            if el.name == "h2" and not content_started:
                text = el.get_text(strip=True)
                if text and not el.find_parent("nav"):
                    main_heading = el
                    content_started = True
                    lvl = int(el.name[1])
                    lines.append(f"{'#' * lvl} {text}\n")
                    break
    
    # 如果找到了主標題，處理它之後的所有元素
    if content_started and main_heading:
        # 找到主標題在文檔中的位置
        start_idx = all_elements.index(main_heading)
        
        # 處理主標題之後的所有元素
        for el in all_elements[start_idx + 1:]:
            if el.name.startswith("h"):
                lvl = int(el.name[1])
                lines.append(f"{'#' * lvl} {el.get_text(strip=True)}\n")
            elif el.name == "p":
                text = el.get_text(strip=True)
                if text:
                    lines.append(text + "\n")
            elif el.name == "pre":
                code = el.get_text()
                lines.append(f"```python\n{code}\n```\n")
            elif el.name == "code":
                inline = el.get_text(strip=True)
                if inline:
                    lines.append(f"`{inline}`\n")
            elif el.name in ("ul", "ol"):
                for li in el.find_all("li"):
                    lines.append(f"- {li.get_text(strip=True)}")
    # 如果無法找到主標題，退回到原始處理方法，但嘗試跳過明顯的導航部分
    else:
        navigation_keywords = ["documentation", "search", "rocket_launch", "Get started", "Develop", "Deploy", "Knowledge base"]
        for el in all_elements:
            skip_element = False
            # 跳過導航相關的元素
            if el.find_parent("nav") or (el.name.startswith("h") and el.get_text(strip=True) in navigation_keywords):
                skip_element = True
                
            if not skip_element:
                if el.name.startswith("h"):
                    lvl = int(el.name[1])
                    lines.append(f"{'#' * lvl} {el.get_text(strip=True)}\n")
                elif el.name == "p":
                    text = el.get_text(strip=True)
                    if text:
                        lines.append(text + "\n")
                elif el.name == "pre":
                    code = el.get_text()
                    lines.append(f"```python\n{code}\n```\n")
                elif el.name == "code":
                    inline = el.get_text(strip=True)
                    if inline:
                        lines.append(f"`{inline}`\n")
                elif el.name in ("ul", "ol"):
                    for li in el.find_all("li"):
                        lines.append(f"- {li.get_text(strip=True)}")

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    with open(save_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    print(f"✅ {save_path}")


def iter_urls(mapping, *parents):
    """
    遞迴把 URL_LIST 攤平，
    mapping: dict or list
    parents: tuple of category, subcategory, …
    回傳 (url, [parents…])
    """
    if isinstance(mapping, dict):
        for key, sub in mapping.items():
            yield from iter_urls(sub, *parents, key)
    elif isinstance(mapping, list):
        for url in mapping:
            yield url, parents
    else:
        raise ValueError("URL_LIST 裡只能放 dict 或 list")


def url_to_filepath(url, categories):
    """
    categories: tuple, e.g. ("api_reference", "text")
    會在 SAVE_ROOT/category1/category2/... 下存 url basename.md
    """
    parsed = urlparse(url)
    name = os.path.basename(parsed.path.rstrip("/")) or "index"
    folder = os.path.join(SAVE_ROOT, *categories)
    os.makedirs(folder, exist_ok=True)
    return os.path.join(folder, f"{name}.md")


def main():
    all_items = list(iter_urls(URL_LIST))  # list of (url, (cat1, cat2, …))
    print(f"🔎 共 {len(all_items)} 個頁面要爬")

    for url, cats in all_items:
        path = url_to_filepath(url, cats)
        try:
            scrape_main_content_as_markdown(url, path)
        except Exception as e:
            print(f"❌ {url} -> {e}")
        time.sleep(CRAWL_DELAY)

    print("🎉 全部抓完！")


if __name__ == "__main__":
    main()


🔎 共 86 個頁面要爬
✅ /Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit/architecture/run-your-app.md
✅ /Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit/architecture/architecture.md
✅ /Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit/architecture/app-chrome.md
✅ /Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit/architecture/caching.md
✅ /Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit/architecture/session-state.md
✅ /Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit/architecture/forms.md
✅ /Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit/architecture/fragments.md
✅ /Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit/architecture/widget-behavior.md
✅ /Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit/multipage/overview.md
✅ /Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit/multipage/page-and-navigation.md
✅ /Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit/multipage/pages-directory.md
✅ /Users/migu/Desktop/資料庫/gen_ai_tr